In [ ]:
from GVAMP_toolbox import *
import pandas as pd

In [ ]:
# parameters

asp_vec = np.linspace(0.1,3,50)   # aspect_ratio
delta0 = 0 # noise (only for linear regression)
rho = 0.3   # sparsity of the teacher
l1=0        # l1 penalty
l2=0.001    # l2 penalty
l2_c = 0    # cooling on l2
damp = 0    # damping paramerer
E_vec=np.zeros(len(asp_vec))    
ang_vec=np.zeros(len(asp_vec))
niter = 400    #maximum number of iterations
tol = 1e-6     # tolerance on the result
d=0            # continuous spectrum or row-orthogonal

In [ ]:
damp = 0
for k in range(0,3):
    asp = asp_vec[k]
    print(k)
    f = lambda p : MP(p,asp)
    E,ang,conv = SE(1,l1,l2,rho,delta0,f,asp,damp,niter,tol,d,0,l2_c) # first number : 0 - linear regression with noisy linear teahcer, 1- loinear regression with noiseless sign teacher, 2-linear SVC with noiseless sign teacher, 3-logistic regression with noiseless sign teacher
    E_vec[k] = E
    ang_vec[k] = ang

In [ ]:
# Experiment

n_exp = 20
asp_vec_n = np.linspace(0.1,3,n_exp) # aspect ratios 
E_vec_n = np.zeros(n_exp)
ang_vec_n = np.zeros(n_exp)
rho = 0.3  # sparsity parameter of the teacher
delta0 = 0
d=200 # low dimension, still gives very good agreement with asymptotics
n_avg = 100 # choose the averaging number
l2 = 0.001
for i in range(n_exp):
    print((i+1)/n_exp*100,'%')
    for k in range(n_avg):  # 20 times averaging, solving the minimization problem with sklearn elastic net
            print(k)
            asp = asp_vec_n[i]
            n = int(asp*d)
            x0 = gauss_bernoulli(rho,d)
            F_train = np.random.normal(0,np.sqrt(1/n),(n,d)) #F_train,D=build_matrix((1-asp)**2,(1+asp)**2,n,d,asp,0)
            w = np.random.normal(0,np.sqrt(delta0),n)
            y_train = np.sign(F_train@x0+w)
            #clf = LinearSVC(loss = 'hinge',tol = 0.00001,C=1/l2,max_iter = 100000)
            #clf = linear_model.ElasticNet(alpha=0.001/n,l1_ratio=1,max_iter = 100000,tol = 0.00001)
            clf = linear_model.LogisticRegression(tol=0.00001,C=1/l2,fit_intercept=False,max_iter=100000)
            clf.fit(F_train,y_train)
            x_train = clf.coef_
            #x_train = np.linalg.inv(np.transpose(F_train)@F_train+l2*np.eye(d))@np.transpose(F_train)@y_train
            E_vec_n[i] = E_vec_n[i]+np.mean((x0-x_train)**2)
            ang_vec_n[i] = ang_vec_n[i]+np.arccos(1/d*x_train@x0/np.sqrt(rho*1/d*np.linalg.norm(x_train)**2))
            
    E_vec_n[i] = 1/n_avg*E_vec_n[i]
    ang_vec_n[i] = 1/n_avg*ang_vec_n[i]

In [ ]:
plt.figure()
plt.title("Square_l2_row_orth_sign_teach")
plt.plot(asp_vec,ang_vec,label="SE_prediction_angle")
plt.plot(asp_vec_n,ang_vec_n,'.',label="experiment_angle")
plt.plot(asp_vec,E_vec,label="SE_prediction_MSE")
plt.plot(asp_vec_n,E_vec_n,'.',label="experiment_MSE")
plt.ylabel("MSE")
plt.xlabel("M/N")
plt.legend()
plt.ylim(0,2)
#plt.savefig('Square_l2_row_orth', dpi=500, quality = 95)

In [ ]:
df = pd.DataFrame(ang_vec, columns=["colummn"])
df.to_csv('Log_l2_0.001_Unif_rho1_ang_SE.csv', index=False)

df = pd.DataFrame(ang_vec_n, columns=["colummn"])
df.to_csv('Log_l2_0.001_Unif_rho1_ang_EXP.csv', index=False)